# Regresión Logística: Detección de SPAM

En este ejercicio se muestran los fundamentos de la Regresión Logística planteando uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de Machine Learning: Detección de SPAM.

## Enunciado del Ejercicio

Se Propone la construcción de un sistema de aprendizaje automático capaz de predicir si un correo determinado corresponde a un correo SPAm o no, para esto, se utilizara el siguiente DataSet.
[DataSet](https://www.kaggle.com/datasets/imdeepmind/preprocessed-trec-2007-public-corpus-dataset)

The corpus trec07p contains 75, 419 messages:

25,220 Ham
50,199 SPAM

These messages contitute all the messages delivered to a particular serever between these dates:

Sun, 8 Apr 2007 13:07:21 -0400
Fri, 6 Jul 2007 07:04:53 -0400

### 1.- Funciones complementarias 

En este caso práctico relacionado con la detección de e-mails de SPAM, el DataSet del que se dispone, esta formado por e-mails, con sus correspondientes cabeceras y campos adicionales. Por lo tanto, requieren un preprocesamiento previo a ser ingeridos por el algoritmo Machine Learning.

In [1]:
# Esta clase facilita el preprocesamiento de correos electrónicos que poseen codigo HTML
from html.parser import HTMLParser
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)


In [2]:
# Esta función se encarga de eliminar los tags HTML que se encuentran en el texto del e-mail
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


In [3]:
#  Ejemplo de eliminación de los tags HTML de un texto
t = '<tr><td align="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>'
strip_tags(t)

'Phrack World News'

Ademas de eliminar los posibles tags HTML que se encuentren en el coreo electrónico, deben realizarse otras acciones de preprocesamiento para evitar que los mensajes tengan ruido inecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, eliminación de posibles campos de correo electrónico que no son reelevantes o eliminación de afijos de una palabra manteniendo únicamente la raiz de la misma (Stemming), La clase que se muestra a continuación realiza estas transformaciones.

In [4]:
import email
import string
import nltk

In [5]:
class Parser:

    def __init__(self):
        self.stemmer =  nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)


    def parse(self, email_path):
        """Parse an email"""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)  
        
    def get_email_content(self, msg):
        """Extract the email content"""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else[]
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return {"subject": subject,
                "body": body,
                "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract the body of the email"""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                   p.get_content_type())
        return body
    
    def tokenize(self, text):
        """Transform a text string in tokens, perform two main action 
        clean the puntuation symbols and do stemming of the text"""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split("")))
        #Stemming of the tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

Lectura de un e-mail en formato Raw 

In [6]:
inmail = open("/home/hernan_so5/Documentos/Ejercicios_jupyter/datasets/datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

Parsear el e-mail

In [ ]:
import nltk
nltk.download('stopwords')
p = Parser()
p.parse("/home/hernan_so5/Documentos/Ejercicios_jupyter/datasets/datasets/trec07p/data/inmail.1")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hernan_so5/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: empty separator